# Sentence classification - Question, Command, Statement

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

## Reading Data and displaying it.
I have processed it already. For you raw data is provided in data file1.csv

In [4]:
import pandas as pd
os.chdir("/content/gdrive/My Drive")

df = pd.read_csv("processed_full_spaadia.csv")
df.head()

,statement,type
0,and confirm that address for me please,command
1,it from birmingham to em london euston please,statement
2,the th of october,statement
3,i like to leave on the train,statement
4,there the from birmingham new street,statement


## Some more exloration

In [5]:
df.shape

(210066, 2)

In [6]:
print(df.isnull().sum())

statement    0
type         0
dtype: int64


## Changing categotical type to labels 

In [0]:
types=df.type.unique()
dic={}
for i,type_ in enumerate(types):
    dic[type_]=i
labels=df.type.apply(lambda x:dic[x])

## Keras Import 

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


## Splitting the data through sklearn

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.statement, df.type, test_size=0.3, random_state=42)

In [0]:
val_data = pd.concat([X_test,y_test],axis=1)
train_data = pd.concat([X_train,y_train],axis=1)

In [0]:
texts=df.statement


## Building Tokenizer

In [12]:
NUM_WORDS=10**5
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(train_data.statement)
sequences_valid=tokenizer.texts_to_sequences(val_data.statement)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 81548 unique tokens.


## Padding

In [0]:
import numpy as np

In [0]:
max_length=50
trunc_type='post'

In [15]:
X_train = pad_sequences(sequences_train,maxlen=max_length, truncating = trunc_type)
X_val = pad_sequences(sequences_valid,maxlen=max_length, truncating = trunc_type)
y_train = to_categorical(np.asarray(labels[train_data.index]))
y_val = to_categorical(np.asarray(labels[val_data.index]))
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (147046, 50) (63020, 50)
Shape of label train and validation tensor: (147046, 3) (63020, 3)


## Constructing the model

In [0]:
from keras.optimizers import Adam
import keras

In [17]:
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
model = keras.Sequential([
    keras.layers.Embedding(vocabulary_size,EMBEDDING_DIM,input_length=max_length),
    keras.layers.LSTM(16, activation='relu',return_sequences=True),
    keras.layers.Dropout(0.25),
    keras.layers.LSTM(8, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(3, activation='softmax')
])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Compiling the model

In [18]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

## Summary

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           24464700  
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 16)            20288     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                288       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)               

## Creating a callback function

In [0]:

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if (logs.get('acc')>logs.get('val_acc')):
      print("\nOverfitting begins")
      self.model.stop_training=True

callbacks = myCallback()

## Fittng the data to model

In [21]:
epochs=10
verbose=1
batch_size=32
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
history=model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_val, y_val), callbacks=[callbacks])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 147046 samples, validate on 63020 samples
Epoch 1/10





147046/147046 [==============================] - 783s 5ms/step - loss: 0.2299 - acc: 0.9199 - val_loss: 0.1371 - val_acc: 0.9532
Epoch 2/10
147046/147046 [==============================] - 764s 5ms/step - loss: 0.1051 - acc: 0.9633 - val_loss: 0.1098 - val_acc: 0.9623

Overfitting begins


## Testing on our own data

In [0]:
test_comments = [
    "This is a stupid example.",
    "This is another statement, perhaps this will trick the network",
    "I don't understand",
    "What's up?",
    "open the app",
    "This is another example",
    "Do what I tell you",
    "come over here and listen",
    "how do you know what to look for",
    "Remember how good the concert was?",
    "Who is the greatest basketball player of all time?",
    "Eat your cereal.",
    "Usually the prior sentence is not classified properly.",
    "Don't forget about your homework!",
    "Can the model identify a sentence without a question mark",
    "Everything speculated here is VC money and financial bubble with unrelaible financial values. Zomato, uber, paytm, flipkart throw discounts at the rate of losses. May be few can survive at the end. This hurts a lot for SMB too.",
    "I am trying to keep tabs on electric two-wheeler startup industry in India. Ather energy is emerging as a big name. Anyone knows how they are doing?",
    "generally a pretty intuitive way to accomplish a task. Want to trash an app Drag it to the trash Want to print a PDF",
    "Make sure ownership is clear and minimizing opportunities for such problematic outcomes in the second place",
    "Stop the video and walk away."
]

test_comments_category = [
    "statement",
    "statement",
    "statement",
    "question",
    "command",
    "statement",
    "command",
    "command",
    "question",
    "question",
    "question",
    "command",
    "statement",
    "command",
    "question",
    "statement",
    "question",
    "question",
    "statement",
    "command"
]

In [0]:
def prediction_single(sent):
    l = []
    l.append(sent)
    sequences_pred = tokenizer.texts_to_sequences(l)
    padded_pred = pad_sequences(sequences_pred,maxlen=max_length, truncating = trunc_type)
    pred = model.predict([padded_pred])
    pred = list(pred[0])
    ind = np.argmax(pred)
    if ind==0:
        return 'command'
    elif ind==1:
        return 'statement'
    else:
        return 'question'

In [25]:
for i in range(len(test_comments)):
    typ = prediction_single(test_comments[i])
    print(test_comments[i],'--------\nActual : ',test_comments_category[i],'--------prediction : ',typ)

This is a stupid example. --------
Actual :  statement --------prediction :  statement
This is another statement, perhaps this will trick the network --------
Actual :  statement --------prediction :  statement
I don't understand --------
Actual :  statement --------prediction :  statement
What's up? --------
Actual :  question --------prediction :  question
open the app --------
Actual :  command --------prediction :  statement
This is another example --------
Actual :  statement --------prediction :  statement
Do what I tell you --------
Actual :  command --------prediction :  question
come over here and listen --------
Actual :  command --------prediction :  statement
how do you know what to look for --------
Actual :  question --------prediction :  question
Remember how good the concert was? --------
Actual :  question --------prediction :  question
Who is the greatest basketball player of all time? --------
Actual :  question --------prediction :  question
Eat your cereal. -------

In [0]:
model.save_weights("processed_full_spaadia_keras_model_new.h5")

In [0]:
import pickle

# saving
with open('processed_full_spaadia_keras_tokenizer_new.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### As you can see there is an imbalance in data i.e, of command type. But it works good on statements and questions. For commands we need more data.